In [16]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning:

Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.



In [17]:
Save_to_html = True

In [18]:
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,10,10,20,20,30,30,40,40,50,50,60,60,70,70,80,80,100]
N_centr = int(len(central_bins)/2)
pt_boarders = [0.2,0.5,0.5,1.0,1.0,5.0]
file_path="input/"
file_names=["my-m_ee_Run14AuAu_74th_new_19784_984runs.root"] 
#m_ee_Run14AuAu_74th_new_19784_984runs
legend_names = ["new_old_x","new_old_y","dca_x","dca_y","vtx_res_x","vtx_res_y","vtx_res_x_reuse","vtx_res_y_reuse", "grid", "nhits"] 
hist_select_3D_names = ["hist_vtx_x","hist_vtx_y","hist_dca_x","hist_dca_y", "hist_vtx_delta_x","hist_vtx_delta_y", "hist_vtx_delta_x_reuse","hist_vtx_delta_y_reuse", "hist_vtx_grid_xy","hits_vtx_ntracks"]

In [19]:
N_hists = len(hist_select_3D_names)

In [20]:
hists_read, hists_sim = [], []

for iFile in range(len(file_names)):
    #print(file_path+file_names[iFile])
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_read_file = []
    for ihist in range(N_hists):
        hists_read_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{iFile}")
    hists_read.append(hists_read_file)


infile.Close()

In [21]:
deltas_mean_x = []
deltas_error_x = []
deltas_mean_y = []
deltas_error_y = []
type_loc = 0
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,3)
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibinz0 = projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX( hists_read[iFile][type_loc].GetName()+f"pos_proj_{icentr}",1,1000,ibinz0,ibinz1)
    y_proj = hists_read[iFile][type_loc+1].ProjectionX( hists_read[iFile][type_loc+1].GetName()+f"y_proj_{icentr}",1,1000,ibinz0,ibinz1)
    deltas_mean_x.append(pos_proj.GetStdDev()*10000)
    deltas_error_x.append(pos_proj.GetStdDevError()*10000)
    deltas_mean_y.append(y_proj.GetStdDev()*10000)
    deltas_error_y.append(y_proj.GetStdDevError()*10000)
    Format_Hist_total(pos_proj,title_x="dx",offset_y=1.1)
    pos_proj.Draw()
    legends.append(root.TLegend(0.2,0.6,0.65,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()

In [22]:
print("Mean of dx")
for i in range(len(deltas_mean_x)):
    print(f"{central_bins[2*i]}-{central_bins[2*i+1]}%: {deltas_mean_x[i]:.3f} +/- {deltas_error_x[i]:.3f}")
    #print(f"{central_bins[2*i]}-{central_bins[2*i+1]}%: {deltas_mean[i]:.3f}")
    #print(f"{central_bins[2*i]}-{central_bins[2*i+1]}%: {deltas_mean[i]:.3f} +/- {deltas_error[i]:.3f}")

Mean of dx
0-10%: 48.771 +/- 0.001
10-20%: 54.754 +/- 0.002
20-30%: 62.321 +/- 0.003
30-40%: 71.471 +/- 0.005
40-50%: 84.371 +/- 0.008
50-60%: 86.667 +/- 0.013
60-70%: 69.568 +/- 0.017
70-80%: 93.654 +/- 0.039
80-100%: 123.724 +/- 0.096


In [23]:
n_points = len(deltas_mean_x)
x_vals = range(n_points)
x_vals_array = array('f', [float(x*10)+5 for x in x_vals])
y_vals_array = array('f', deltas_mean_x)
x_errs_array = array('f', [0.0]*n_points)
y_errs_array = array('f', deltas_error_x)


x1_vals_array = array('f', [float(x*10)+5 for x in x_vals])
y1_vals_array = array('f', deltas_mean_y)
x1_errs_array = array('f', [0.0]*n_points)
y1_errs_array = array('f', deltas_error_y)

g = root.TGraphErrors(n_points, x_vals_array, y_vals_array, x_errs_array, y_errs_array)
g.SetTitle("Mean and Error;Centrality bin;StdDev [#mum]")
g.SetMarkerStyle(20)
g.SetMarkerColor(root.kBlue)
g.SetLineColor(root.kBlue)
g.SetMinimum(0)


g1 = root.TGraphErrors(n_points, x1_vals_array, y1_vals_array, x1_errs_array, y1_errs_array)
g1.SetMarkerStyle(20)
g1.SetMarkerColor(root.kRed)
g1.SetLineColor(root.kRed)

c = root.TCanvas("c_deltas", "deltas_mean with error", 800, 600)
g.Draw("AP")    
g1.Draw("P")
c.Draw()

In [24]:
deltas_mean_x = []
deltas_error_x = []
deltas_mean_y = []
deltas_error_y = []
type_loc = 4
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibinz0 = projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX( hists_read[iFile][type_loc].GetName()+f"pos_proj_{icentr}",1,1000,ibinz0,ibinz1)
    y_proj = hists_read[iFile][type_loc+1].ProjectionX( hists_read[iFile][type_loc+1].GetName()+f"y_proj_{icentr}",1,1000,ibinz0,ibinz1)
    deltas_mean_x.append(pos_proj.GetStdDev()*10000/2**0.5)
    deltas_error_x.append(pos_proj.GetStdDevError()*10000)
    deltas_mean_y.append(y_proj.GetStdDev()*10000/2**0.5)
    deltas_error_y.append(y_proj.GetStdDevError()*10000)

In [25]:
n_points = len(deltas_mean_x)
x_vals = range(n_points)
x_vals_array = array('f', [float(x*10)+5 for x in x_vals])
y_vals_array = array('f', deltas_mean_x)
x_errs_array = array('f', [0.0]*n_points)
y_errs_array = array('f', deltas_error_x)


x1_vals_array = array('f', [float(x*10)+5 for x in x_vals])
y1_vals_array = array('f', deltas_mean_y)
x1_errs_array = array('f', [0.0]*n_points)
y1_errs_array = array('f', deltas_error_y)

g = root.TGraphErrors(n_points, x_vals_array, y_vals_array, x_errs_array, y_errs_array)
g.SetTitle("Mean and Error;Centrality bin;StdDev [#mum]")
g.SetMarkerStyle(20)
g.SetMarkerColor(root.kBlue)
g.SetLineColor(root.kBlue)
g.SetMinimum(0)

g1 = root.TGraphErrors(n_points, x1_vals_array, y1_vals_array, x1_errs_array, y1_errs_array)
g1.SetMarkerStyle(20)
g1.SetMarkerColor(root.kRed)
g1.SetLineColor(root.kRed)

c = root.TCanvas("c_deltas", "deltas_mean with error", 800, 600)
g.Draw("AP")    
g1.Draw("P")
c.Draw()

In [26]:
deltas_mean_x = []
deltas_error_x = []
deltas_mean_y = []
deltas_error_y = []
type_loc = 6
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibinz0 = projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX( hists_read[iFile][type_loc].GetName()+f"pos_proj_{icentr}",1,1000,ibinz0,ibinz1)
    y_proj = hists_read[iFile][type_loc+1].ProjectionX( hists_read[iFile][type_loc+1].GetName()+f"y_proj_{icentr}",1,1000,ibinz0,ibinz1)
    deltas_mean_x.append(pos_proj.GetStdDev()*10000)
    deltas_error_x.append(pos_proj.GetStdDevError()*10000)
    deltas_mean_y.append(y_proj.GetStdDev()*10000)
    deltas_error_y.append(y_proj.GetStdDevError()*10000)

In [27]:
n_points = len(deltas_mean_x)
x_vals = range(n_points)
x_vals_array = array('f', [float(x*10)+5 for x in x_vals])
y_vals_array = array('f', deltas_mean_x)
x_errs_array = array('f', [0.0]*n_points)
y_errs_array = array('f', deltas_error_x)


x1_vals_array = array('f', [float(x*10)+5 for x in x_vals])
y1_vals_array = array('f', deltas_mean_y)
x1_errs_array = array('f', [0.0]*n_points)
y1_errs_array = array('f', deltas_error_y)

g = root.TGraphErrors(n_points, x_vals_array, y_vals_array, x_errs_array, y_errs_array)
g.SetTitle("Mean and Error;Centrality bin;StdDev [#mum]")
g.SetMarkerStyle(20)
g.SetMarkerColor(root.kBlue)
g.SetLineColor(root.kBlue)
g.SetMinimum(0)


g1 = root.TGraphErrors(n_points, x1_vals_array, y1_vals_array, x1_errs_array, y1_errs_array)
g1.SetMarkerStyle(20)
g1.SetMarkerColor(root.kRed)
g1.SetLineColor(root.kRed)

c = root.TCanvas("c_deltas", "deltas_mean with error", 800, 600)
g.Draw("AP")    
g1.Draw("P")
c.Draw()

In [28]:
deltas_mean_x = []
deltas_error_x = []
deltas_mean_y = []
deltas_error_y = []
type_loc = 9
c0 = root.TCanvas(f"c0",f"c0",1450,900)
hists_read[0][type_loc].Draw("colz")
root.gPad.SetLogz()
c0.Draw()

In [29]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming hist2d is already defined, e.g.:
hist2d = hists_read[0][type_loc]

# Extract bin info
nbins_x = hist2d.GetNbinsX()
nbins_y = hist2d.GetNbinsY()

# Axis bin centers
x_centers = [hist2d.GetXaxis().GetBinCenter(i+1) for i in range(nbins_x)]
y_centers = [hist2d.GetYaxis().GetBinCenter(j+1) for j in range(nbins_y)]

# Z content: shape (nbins_y, nbins_x) for Plotly
z_content = np.array([[hist2d.GetBinContent(i+1, j+1) 
                       for i in range(nbins_x)] for j in range(nbins_y)])

# Projection onto X: sum along Y for each X bin
proj_x = np.sum(z_content, axis=0)

# --- Create subplots: 2 rows ---
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,  # ✅ correct
    row_heights=[0.7, 0.3],
    vertical_spacing=0.05,
    subplot_titles=("2D Histogram", "Projection onto X-axis")
)


# Add 2D heatmap
fig.add_trace(
    go.Heatmap(
        x=x_centers,
        y=y_centers,
        z=z_content,
        coloraxis="coloraxis"
    ),
    row=1, col=1
)

# Add 1D X projection
fig.add_trace(
    go.Bar(
        x=x_centers,
        y=proj_x,
        name="X Projection"
    ),
    row=2, col=1
)

# Layout
fig.update_layout(
    height=700,
    width=900,
    coloraxis={"colorscale": "Viridis"},
    xaxis2_title="X axis",
    yaxis1_title="Y axis",
    yaxis2_title="Counts",
    showlegend=False
)

fig.show()


In [30]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact
import ipywidgets as widgets

# -- Your TH2 histogram from ROOT
hist2d = hists_read[0][type_loc]
nbins_x = hist2d.GetNbinsX()
nbins_y = hist2d.GetNbinsY()

# Bin centers
x_centers = [hist2d.GetXaxis().GetBinCenter(i+1) for i in range(nbins_x)]
y_centers = [hist2d.GetYaxis().GetBinCenter(j+1) for j in range(nbins_y)]

# Full Z data for 2D heatmap
z_content = np.array([[hist2d.GetBinContent(i+1, j+1)
                       for i in range(nbins_x)] for j in range(nbins_y)])

# -- Draw 2D heatmap once
heatmap_fig = go.Figure()
heatmap_fig.add_trace(go.Heatmap(
    x=x_centers,
    y=y_centers,
    z=z_content,
    coloraxis="coloraxis"
))
heatmap_fig.update_layout(
    height=500,
    width=800,
    title="2D Histogram",
    coloraxis={"colorscale": "Viridis"},
    xaxis_title="X",
    yaxis_title="Y"
)
heatmap_fig.show()

# -- Interactive projection
@interact(binY=widgets.IntSlider(min=1, max=nbins_y, step=1, value=1, description='Y bin'))
def update_projection(binY):
    binY_index = binY - 1
    projection = z_content[binY_index, :]  # Single row = X projection at Y bin
    y_bin_center = y_centers[binY_index]

    fig = go.Figure()
    fig.add_trace(go.Bar(x=x_centers, y=projection))
    fig.update_layout(
        title=f"X Projection at Y = {y_bin_center:.2f}",
        xaxis_title="X",
        yaxis_title="Counts",
        height=300,
        width=800
    )
    fig.show()


interactive(children=(IntSlider(value=1, description='Y bin', max=10, min=1), Output()), _dom_classes=('widget…